In [6]:
import pandas as pd
import pickle 
#from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
#from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
#from finrl.meta.data_processor import DataProcessor
import sys
sys.path.append("/findrl_ray/finenv")
sys.path.append("./FinRL")
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import data_split
# load the DataFrame from a pickle file
#df = pd.read_pickle('dataset/nasdaq74_proc_test.pkl')

train = pd.read_csv('dataset/train_data.csv')
train = train.set_index(train.columns[0])
train.index.names = ['']

TRAIN_START_DATE = '2016-01-01'
TRAIN_END_DATE = '2018-01-01'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2023-03-26'
#df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [7]:
INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']

In [8]:
#train = data_split(df, TRAIN_START_DATE,TRAIN_END_DATE)
#trade = data_split(df, TRADE_START_DATE,TRADE_END_DATE)
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 80, State Space: 801


In [9]:
train.head()
train.dtypes

date             object
tic              object
open            float64
high            float64
low             float64
close           float64
volume          float64
day             float64
macd            float64
boll_ub         float64
boll_lb         float64
rsi_30          float64
cci_30          float64
dx_30           float64
close_30_sma    float64
close_60_sma    float64
dtype: object

In [12]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
#env_kwargs = {
#    "hmax": 500,
#    "initial_amount": 1000000,
#    "num_stock_shares": num_stock_shares,
#    "buy_cost_pct": buy_cost_list,
 #   "sell_cost_pct": sell_cost_list,
  #  "state_space": state_space,
   # "stock_dim": stock_dimension,
   # "tech_indicator_list": INDICATORS,
   # "action_space": stock_dimension,
   # "reward_scaling": 1e-4
#}

In [13]:
import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility

In [22]:
def env_creator(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', train)
    hmax = env_config.get('hmax', 200)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-2)

    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling
    ))

In [27]:
from ray.rllib.agents import ppo
ray.shutdown()
print(f"ray is being initialized")
# ray.init(_temp_dir="FinRL/RLLIB/results", num_cpus=1, num_gpus=0)
# ray.init()

ray is being initialized


In [10]:
#if use_ddppo:
#    config = ppo.DDPPOConfig()
#else:
#    config = ppo.PPOConfig()

# Here set envirnonment parameters if different from default
#config = config.environment(
#    env_config={'hmax':200, 'initial_amount':1000000}
#)

# Training
#config = config.training()

# Resources
#config = config.resources(
#    num_gpus=0,
#    num_cpus_per_worker=1,
#    num_gpus_per_worker=0,
#    num_trainer_workers=1,
#    num_gpus_per_trainer_worker = 1,
#    num_cpus_per_trainer_worker = 1,
#)
# Framework
#config = config.framework(
#    framework="torch",
#)
# Rollouts
#config = config.rollouts(
#    num_rollout_workers=10,
#    num_envs_per_worker=1,
#)

In [24]:
config = ppo.PPOConfig()
config = config.environment(env_config={'hmax':500,'initial_amount':300000})
config = config.training(gamma=0.9, lr=0.0001, kl_coeff=0.3)  
config = config.resources(num_gpus=1)  
config = config.rollouts(num_rollout_workers=15) 
config = config.framework(framework="torch")
config["model"]["fcnet_hiddens"] = [1024, 256, 128,32]

In [25]:
# registering the environment to ray
register_env("finrl", env_creator)
trainer = config.build(env="finrl") 
if use_ddppo:
    trainer = ppo.DDPPOTrainer(env='finrl', config=config)
    #trainer = ppo.PPOTrainer(env='finrl', config=config)
    

2023-04-01 22:59:58,149	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
(RolloutWorker pid=12200) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=12200)   logger.warn("Casting input x to numpy array.")
(RolloutWorker pid=12145) 2023-04-01 23:00:12,593	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=12145) 2023-04-01 23:00:12,594	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=12145) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN

In [26]:
from tqdm.notebook import tqdm
# Train away -------------------------------------------------------------
total_episodes=20
agent_name = 'ppo'
ep = 0
results = []
bar = tqdm(total=total_episodes, desc="Episodes")
while ep <= total_episodes:
    results.append(trainer.train())
    ep += 1
    bar.update(n=1)
    if ep % 10 == 0:
        #cwd_checkpoint = "results/checkpoints/" + str(agent_name) + '_' + str(ep)
        cwd_checkpoint = "model/" + str('4fcnet')
        trainer.save(cwd_checkpoint)
        print(f"Checkpoint saved in directory {cwd_checkpoint}")
bar.close()

Episodes:   0%|          | 0/20 [00:00<?, ?it/s]

2023-04-01 23:00:48,645	ERROR actor_manager.py:496 -- Ray error, taking actor 15 out of service. The actor died unexpectedly before finishing this task.
(raylet) [2023-04-01 23:00:58,141 E 10929 10929] (raylet) node_manager.cc:3040: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 3ca2d4fe9094ac924c7441b9bd79a076543bb295876f11121da397b3, IP: 172.28.230.93) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.28.230.93`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RA

KeyboardInterrupt: 

In [14]:
## Mean Rewards ### 
print(results[-1]['episode_reward_mean'])
#print(results[-1])

690.8734103703702


In [15]:
# save the trained agent
#cwd_checkpoint = "model/" + str(agent_name) + '_' + str(ep)
cwd_checkpoint = "model/"
trainer.save(cwd_checkpoint)

'model/checkpoint_000021'